In [1]:
# @TODO: try this for y-values
# sometimes this is called “non-parametric” in classical statistics
# rank-based /percentile cutoff

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

from sklearn.cross_validation import train_test_split, StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
                            accuracy_score, 
                            precision_score, 
                            recall_score, 
                            f1_score, 
                            roc_curve, 
                            roc_auc_score, 
                            confusion_matrix
                            )
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.learning_curve import learning_curve
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import confusion_matrix

from unbalanced_dataset import OverSampler, SMOTE

import matplotlib.pyplot as plt
# import seaborn as sns

import statsmodels.api as sm
from statsmodels.tsa.ar_model import AR, ARResults
from scipy.misc import factorial as fact

%matplotlib inline

In [2]:
CONFIG = {}

with open('config.txt', "r") as in_file:
    for line in in_file:
        line = line.split(":")
        parameter = line[0].strip()
        value = line[1].strip()
        CONFIG[parameter] = value
                
engine_path = 'postgresql://'+CONFIG['username']+':'+\
                CONFIG['password']+'@'+CONFIG['ip']+\
                ':'+CONFIG['port']+'/'+CONFIG['database']
        
engine = create_engine(engine_path)

##Want something like:

In [3]:
def makeSQLQuery(table_names, year, seriesnames):
    dotyyear = '.y' + str(year)
    short = table_names[0][0:2]
    seriesnames = seriesnames
    abb = [ser[0:2] for ser in seriesnames]
    query = 'SELECT x.*, y' + dotyyear + ' as zscores FROM (' + \
            'SELECT ' + short + '.countrycode AS countrycode, ' + \
            short + dotyyear + ' AS ' + table_names[0] 
    
    mystr = ''
    for s in seriesnames:
        mystr = mystr + ', env.' + s
    mystr = mystr
    
    query = query + mystr
    
    query = query + ' FROM (SELECT countrycode, ' + dotyyear[1:] + \
        ' FROM ' + table_names[0] + ') ' + short + ' JOIN ' + \
        '(SELECT ' + abb[0] + '.countrycode AS countrycode'
    
    mystr = ''
    for a, ser in zip(abb, seriesnames):
        mystr = mystr + ', ' + a + dotyyear + ' as ' + ser
    query = query + mystr
    
    query = query + ' FROM ('
    
    subquery = []
    for a, ser in zip(abb, seriesnames):
        mystr = ''
        mystr = mystr + '(SELECT countrycode, ' + dotyyear[1:] + \
        ' FROM ' + table_names[1] + " WHERE series='" + ser + \
        "') " + a
        subquery.append(mystr)
    
    query = query + subquery[0] + ' JOIN ' + subquery[1] + ' ON ' + \
        abb[0] + '.countrycode = ' + abb[1] + '.countrycode' + \
        ' JOIN ' + subquery[2] + ' ON ' + \
        abb[0] + '.countrycode = ' + abb[2] + '.countrycode' + \
        ' JOIN ' + subquery[3] + ' ON ' + \
        abb[0] + '.countrycode = ' + abb[3] + '.countrycode)) env' + \
        ' ON ' + short + '.countrycode = env.countrycode'
        
    query = query + ') x JOIN (select countrycode, ' + dotyyear[1:] + \
            ' FROM zscores) y on x.countrycode = y.countrycode'
    
    return query

In [4]:
years = [x for x in range(2006, 2014)]

In [5]:
tables = ['forestarea', 'environment']
features = ['precipitation', 'extreme', 'arable_land', 'freshwater']
print makeSQLQuery(tables, years[0], features)

SELECT x.*, y.y2006 as zscores FROM (SELECT fo.countrycode AS countrycode, fo.y2006 AS forestarea, env.precipitation, env.extreme, env.arable_land, env.freshwater FROM (SELECT countrycode, y2006 FROM forestarea) fo JOIN (SELECT pr.countrycode AS countrycode, pr.y2006 as precipitation, ex.y2006 as extreme, ar.y2006 as arable_land, fr.y2006 as freshwater FROM ((SELECT countrycode, y2006 FROM environment WHERE series='precipitation') pr JOIN (SELECT countrycode, y2006 FROM environment WHERE series='extreme') ex ON pr.countrycode = ex.countrycode JOIN (SELECT countrycode, y2006 FROM environment WHERE series='arable_land') ar ON pr.countrycode = ar.countrycode JOIN (SELECT countrycode, y2006 FROM environment WHERE series='freshwater') fr ON pr.countrycode = fr.countrycode)) env ON fo.countrycode = env.countrycode) x JOIN (select countrycode, y2006 FROM zscores) y on x.countrycode = y.countrycode


In [13]:
df = pd.DataFrame()

In [18]:
for year in years:
    query = makeSQLQuery(tables, year, features)
    year_data = pd.read_sql_query(query, engine)
    year_data['year'] = year
    year_data = year_data.set_index(['countrycode', 'year'])
    df = pd.concat([df, year_data])

In [21]:
df.columns

Index([u'forestarea', u'precipitation', u'extreme', u'arable_land',
       u'freshwater', u'zscores'],
      dtype='object')

In [16]:
df.size

7506

In [24]:
df.head()

,,forestarea,precipitation,extreme,arable_land,freshwater,zscores
countrycode,year,,,,,,
ARG,2006,10.913914,591,0.167190,12.735823,7305.496838,-0.278010269
AUS,2006,16.499616,534,3.047099,6.211031,23622.500912,-0.279337781
BRA,2006,60.429569,1761,0.482451,8.319794,29364.390723,-0.27919335
CHN,2006,20.723562,645,7.950971,11.578350,2134.480626,-0.272709544
FRA,2006,29.172541,867,0.005718,33.490456,3124.207769,-0.279344809


In [27]:
df['zscores'] = df['zscores'].astype(float)

In [29]:
df = df.dropna()

In [30]:
df.describe()

,forestarea,precipitation,extreme,arable_land,freshwater,zscores
count,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000
mean,30.606953,1112.219025,1.214430,16.109750,14878.542330,0.040514
std,22.133838,757.134960,1.976777,13.874216,35020.530613,1.143605
min,0.067909,51.000000,0.000000,0.051769,0.000000,-0.293136
25%,11.078235,562.000000,0.023082,5.127726,1147.784317,-0.276353
50%,30.513576,1010.000000,0.269239,11.936709,3063.458600,-0.262625
75%,46.556322,1604.000000,1.295973,22.676412,12391.666391,-0.197219
max,98.506410,3240.000000,9.226586,60.536222,323213.140961,9.868634


In [36]:
df['binary'] = df['zscores'].apply(lambda x: 0 if x<0 else 1)

/Users/emilyhoughkovacs/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [39]:
len(df[df['binary']==1])

394

In [40]:
len(df)

2502

In [41]:
394./2502*100

15.74740207833733

In [42]:
df.describe()

,forestarea,precipitation,extreme,arable_land,freshwater,zscores,binary
count,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000
mean,30.606953,1112.219025,1.214430,16.109750,14878.542330,0.040514,0.157474
std,22.133838,757.134960,1.976777,13.874216,35020.530613,1.143605,0.364320
min,0.067909,51.000000,0.000000,0.051769,0.000000,-0.293136,0.000000
25%,11.078235,562.000000,0.023082,5.127726,1147.784317,-0.276353,0.000000
50%,30.513576,1010.000000,0.269239,11.936709,3063.458600,-0.262625,0.000000
75%,46.556322,1604.000000,1.295973,22.676412,12391.666391,-0.197219,0.000000
max,98.506410,3240.000000,9.226586,60.536222,323213.140961,9.868634,1.000000


In [43]:
df = df.swaplevel(0, 1, axis=0)

In [150]:
df.head()

forestarea  precipitation   extreme  arable_land  \
year countrycode                                                     
2006 ARG           10.913914            591  0.167190    12.735823   
     AUS           16.499616            534  3.047099     6.211031   
     BRA           60.429569           1761  0.482451     8.319794   
     CHN           20.723562            645  7.950971    11.578350   
     FRA           29.172541            867  0.005718    33.490456   

                    freshwater  zscores  
year countrycode                         
2006 ARG           7305.496838        0  
     AUS          23622.500912        0  
     BRA          29364.390723        0  
     CHN           2134.480626        0  
     FRA           3124.207769        0

In [44]:
x_cols = ['forestarea', 'precipitation', 'extreme', 'arable_land', 'freshwater']

In [45]:
# convert each feature to z-score to normalize
for col in x_cols:
    df[col] = (df[col] - df[col].mean())/df[col].std(ddof=0)

In [150]:
def getScoreValues(X_train, 
                   X_test, 
                   y_train, 
                   y_test, 
                   model
                  ):
    model = model
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    accuracy_score_value = accuracy_score(y_test, y_pred)
    precision_score_value = precision_score(y_test, y_pred)
    recall_score_value = recall_score(y_test, y_pred)
    f1_score_value = f1_score(y_test, y_pred) 
    
    scores = pd.Series(data=[accuracy_score_value, precision_score_value, recall_score_value, f1_score_value],
                      index=["Accuracy", "Precision", "Recall", "F1"])
    
    print model
    print scores
    return y_pred

In [109]:
def getFeatureImportance(X_train, 
                   X_test, 
                   y_train, 
                   y_test, 
                   model, 
                   verbose=True, 
                   get_features=True
                  ):
    model = model
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    try:
        feature_importances = model.feature_importances_
        #print(feature_importances)
    except AttributeError:
        pass
    fimp = pd.Series(data=feature_importances, index=x_cols)
    return fimp

In [49]:
X = df[x_cols]
y = df['binary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state=4444)

In [108]:
getScoreValues(X_train, X_test, y_train, y_test, DecisionTreeClassifier())

Accuracy     0.985353
Precision    0.950413
Recall       0.958333
F1           0.954357
dtype: float64

In [110]:
getFeatureImportance(X_train, X_test, y_train, y_test, DecisionTreeClassifier())

forestarea       0.302181
precipitation    0.312228
extreme          0.057418
arable_land      0.169783
freshwater       0.158390
dtype: float64

In [112]:
test = np.arange(min(years)+1, max(years))
train = []
for y in test:
    tr = [yr for yr in years if yr<y]
    train.append(tr)
    print tr, y
    
training_sets = []
test_sets = []


[2006] 2007
[2006, 2007] 2008
[2006, 2007, 2008] 2009
[2006, 2007, 2008, 2009] 2010
[2006, 2007, 2008, 2009, 2010] 2011
[2006, 2007, 2008, 2009, 2010, 2011] 2012


In [113]:
for prev, year in zip(train, test):
    test_sets.append(df.xs(year, level='year', axis=0))
    
    for year in test:
        trainyear = pd.DataFrame()
        for y in prev:
            data = df.xs(y, level='year', axis=0)
            trainyear = pd.concat([trainyear, data])
    
    training_sets.append(trainyear)

In [139]:
fa, pr, ex, ar, fr = 0, 0, 0, 0, 0
for tr, te in zip(training_sets, test_sets):
    results = getFeatureImportance(tr[x_cols], te[x_cols], tr['binary'], te['binary'], DecisionTreeClassifier())
    fa += results[0]
    pr += results[1]
    ex += results[2]
    ar += results[3]
    fr += results[4]
fimp = [x/len(test_sets) for x in [fa, pr, ex, ar, fr]]
print fimp

[0.21864933972542758, 0.24401846069668864, 0.16240952023634703, 0.17408796363568399, 0.20083471570585273]


In [140]:
def score_model(train, test, model):
    year = 2007
    dfscores = pd.DataFrame()
    for train, test in zip(train, test):
        X_train, X_test, y_train, y_test = train[x_cols], test[x_cols], train['binary'], test['binary']
        scores = getScoreValues(X_train, X_test, y_train, y_test, DecisionTreeClassifier())
        dfscores = dfscores.append(scores, ignore_index=True)
        year += 1
    avgscores = [(s, dfscores[s].mean()) for s in dfscores]
    totscores = pd.Series([dfscores[s].mean() for s in dfscores], index=[s for s in dfscores])
#     print avgscores
    return totscores

In [180]:
models = [DecisionTreeClassifier(), BernoulliNB(), GaussianNB(), 
          SVC(probability=True), RandomForestClassifier(), KNeighborsClassifier()]

In [59]:
modelnames = ["Decision Tree", "Bernoulli NB", "Gaussian NB", "SVM", "Random Forest", "K Neighbors"]

In [141]:
dfmodelscores = pd.DataFrame(index=["Accuracy", "Precision", "Recall", "F1"], columns=modelnames)
for model, name in zip(models, modelnames):
    print model
    print score_model(training_sets, test_sets, model)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best')
Accuracy     0.963716
F1           0.885137
Precision    0.898889
Recall       0.873258
dtype: float64
BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)
Accuracy     0.965860
F1           0.892977
Precision    0.900449
Recall       0.887449
dtype: float64
GaussianNB()
Accuracy     0.966921
F1           0.895953
Precision    0.905833
Recall       0.887449
dtype: float64
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
Accuracy     0.965860
F1           0.892876
Precision    0.901474
Recall       0.886613
dtype: float64
RandomForestClassifier(bootstrap=True, class_weight=None

In [67]:
smallxcols = ['forestarea', 'precipitation', 'freshwater']

In [93]:
def score_model(train, test, model):
    year = 2007
    dfscores = pd.DataFrame()
    for train, test in zip(train, test):
        X_train, X_test, y_train, y_test = train[smallxcols], test[smallxcols], train['binary'], test['binary']
        scores = getScoreValues(X_train, X_test, y_train, y_test, DecisionTreeClassifier())
        dfscores = dfscores.append(scores, ignore_index=True)
        year += 1
    avgscores = [(s, dfscores[s].mean()) for s in dfscores]
    totscores = pd.Series([dfscores[s].mean() for s in dfscores], index=[s for s in dfscores])
#     print avgscores
    return totscores

In [82]:
type(score_model(training_sets, test_sets, DecisionTreeClassifier()))

pandas.core.series.Series

In [97]:
dfmodelscoresSmall = pd.DataFrame(index=["Accuracy", "Precision", "Recall", "F1"], columns=modelnames)
for model, name in zip(models, modelnames):
    print name
    print score_model(training_sets, test_sets, model)

Decision Tree
Accuracy     0.961579
F1           0.879918
Precision    0.886166
Recall       0.879668
dtype: float64
Bernoulli NB
Accuracy     0.957319
F1           0.867667
Precision    0.872194
Recall       0.866848
dtype: float64
Gaussian NB
Accuracy     0.956251
F1           0.863618
Precision    0.871166
Recall       0.860438
dtype: float64
SVM
Accuracy     0.958381
F1           0.871334
Precision    0.874817
Recall       0.873258
dtype: float64
Random Forest
Accuracy     0.959456
F1           0.873727
Precision    0.877941
Recall       0.873258
dtype: float64
K Neighbors
Accuracy     0.958381
F1           0.871447
Precision    0.875378
Recall       0.873258
dtype: float64


In [152]:
len(training_sets)

6

In [154]:
y_pred = getScoreValues(training_sets[5][x_cols], test_sets[5][x_cols], 
                        training_sets[5]['binary'], test_sets[5]['binary'], DecisionTreeClassifier())

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best')
Accuracy     0.974359
Precision    0.958333
Recall       0.884615
F1           0.920000
dtype: float64


In [159]:
y_actual = np.array(test_sets[5]['binary'])

In [166]:
diff = y_actual-y_pred

In [168]:
diff = pd.Series(diff)

In [177]:
results = pd.DataFrame({'y_actual': y_actual, 'y_pred': y_pred, 'diff': y_actual-y_pred}, index=test_sets[5]['binary'].index)

In [178]:
results[results['diff']!=0]

,diff,y_actual,y_pred
countrycode,,,
AGO,-1,0,1
HTI,1,1,0
MLI,1,1,0
SYR,1,1,0
AGO,-1,0,1
HTI,1,1,0
MLI,1,1,0
SYR,1,1,0


In [160]:
y_actual

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 0,

In [145]:
model

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_neighbors=5, p=2, weights='uniform')

In [236]:
def getROCcurve(X_train, X_test, y_train, y_test, model):
    model = model
    model.fit(X_train, y_train)
    y_scores = model.predict_proba(X_test)[:,1]
    fpr, tpr, thresholds = roc_curve(y_test, y_scores)

    auc = roc_auc_score(y_test, y_scores)
#     print('AUC: {}'.format(auc))
    
    fig,ax = plt.subplots()
    ax.plot(fpr, tpr, label='AUC: %0.3f' % auc)

    fig.set_size_inches(12, 8, forward=True)
    plt.xlabel('False Postive Rate', fontsize=14)
    plt.ylabel('True Positive Rate', fontsize=14)
    plt.title('Random Forest ROC Curve for Climate-Triggered Refugee Disasters', fontsize=16)
    plt.legend(loc='best', fontsize=14)

In [181]:
def getROCcurve2(X_train, X_test, y_train, y_test, models):
    for m in models:
        model.fit(X_train, y_train)
        y_scores = model.predict_proba(X_test)[:,1]
        fpr, tpr, thresholds = roc_curve(y_test, y_scores)

        auc = roc_auc_score(y_test, y_scores)
    #     print('AUC: {}'.format(auc))

        fig,ax = plt.subplots()
        ax.plot(fpr, tpr, label='AUC: %0.3f' % auc)

    fig.set_size_inches(12, 8, forward=True)
    plt.xlabel('False Postive Rate', fontsize=14)
    plt.ylabel('True Positive Rate', fontsize=14)
    plt.title('Random Forest ROC Curve for Climate-Triggered Refugee Disasters', fontsize=16)
    plt.legend(loc='best', fontsize=14)

In [257]:
test2013 = df.xs(2013, level='year', axis=0)

In [261]:
train2013 = pd.concat([training_sets[5], df.xs(2012, level='year', axis=0)])

In [262]:
X_train, X_test, y_train, y_test = train2013[x_cols], test2013[x_cols], train2013['zscores'], test2013['zscores']